In [1]:
import simpy
import random
import numpy as np
from simpy.events import Event

In [2]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

SHIFT = 480

ANSWERED_CALLS = 1000
FINISHED_T = 0
all_customers = []
operation_t = [0,0]
shift_number = 0
is_done = 0

In [3]:
def service(env, opr):
    if opr==operator1:
        time = random.lognormvariate(M,S)
        operation_t[0] += time
        yield env.timeout(time)
        
    elif opr==operator2:
        time = random.uniform(*service_range)
        operation_t[1] += time
        yield env.timeout(time)

In [4]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.waiting_t = 0
        
    
    def call(self):
        global is_done
        print('%s initiated a call at %g' % (self.name, self.env.now))
        
        
        # Voice recognition and routing
        if router.is_idle():
            router.working_t += [self.env.now]    
        router.count += [self.name]
        
        yield self.env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
        router.count.remove(self.name)
        if router.is_idle():
            router.idle_t += [self.env.now]
        
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
            print('%s hangs up the call at %g' % (self.name, self.env.now))
            #all_customers.append(self)
            self.operator = None
            is_done += 1
            if is_done == ANSWERED_CALLS:
                finished.succeed()
            return 
        
        print('%s is rooted to operator %d at %g' % (self.name,(self.operator==operator2)+1, self.env.now))
        
        with self.operator.request(0) as req:
            self.waiting_t = self.env.now
            result = yield req | env.timeout(10)
            # Reneging for 10 mins
            if req not in result:
                print('%s is left the operator %d after 10 minutes at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
                #all_customers.append(self)
                self.operator = None
                is_done += 1
                if is_done == ANSWERED_CALLS:
                    finished.succeed()
                return 
            self.waiting_t = self.env.now - self.waiting_t
            print('%s is assigned to the operator %d at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
            yield self.env.process(service(env,self.operator))
            print('%s is done with operator %d at %g' % (self.name,
                                                        (self.operator==operator2)+1,
                                                        self.env.now))
            #all_customers.append(self)
            is_done += 1
            if is_done == ANSWERED_CALLS:
                finished.succeed()

In [5]:
class robocall:
    def __init__(self,env):
        self.count = []
        self.working_t = []
        self.idle_t = [0]
        self.env = env
        
    def answer(self):
        return len(self.count) < 100
    
    def is_idle(self):
        return len(self.count) == 0
            
    def utilization(self):
        return sum([ w-i for i,w in zip(router.idle_t, router.working_t)]) / FINISHED_T

In [6]:
def breaks():
    return [random.uniform(0,SHIFT) for i in  range(np.random.poisson(8))]

In [7]:
def shift(env):
    global shift_number
    shift_number += 1
    
    [env.process(take_a_break(env, b, operator1)) for b in breaks()]
    [env.process(take_a_break(env, b, operator2)) for b in breaks()]

    yield env.timeout(SHIFT)
    print("the time %g" % env.now)

    env.process(shift(env))

In [8]:
def take_a_break(env, number, opr):
    yield env.timeout(number)
    with opr.request(1) as brk:
        yield brk
        
        if SHIFT*shift_number - env.now > 0:
            if SHIFT*shift_number - env.now >= 3:
                print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
                yield env.timeout(3)
            else:
                print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
                yield env.timeout(480*shift_number - env.now)
            print("break ended for operator %d at %g" % ((opr == operator2)+1 , env.now))

In [9]:
def customer_generator(env):
    start = True
    num_answered = 0;
    
    while(len(all_customers)<ANSWERED_CALLS):
        
        #Every 480 a new shift starts.
        if start:
            env.process(shift(env))
            start = False
            
        yield env.timeout(random.expovariate(1/interarrival_mean))
        
        # Voice recognition system limit
        if router.answer():
            num_answered += 1
            customer = Customer('Customer %s' %(num_answered), env)
            all_customers.append(customer)
            
    FINISHED_T = env.now 

In [10]:
env = simpy.Environment()

finished = Event(env)
router = robocall(env)

operator1 = simpy.PriorityResource(env, capacity = 1)
operator2 = simpy.PriorityResource(env, capacity = 1)

env.process(customer_generator(env))
env.run(finished) 
FINISHED_T = env.now
print(FINISHED_T)
print(len(all_customers))

Customer 1 initiated a call at 26.5136
Customer 2 initiated a call at 27.7602
Customer 3 initiated a call at 29.5965
Customer 2 is rooted to operator 2 at 30.0202
Customer 2 is assigned to the operator 2 at 30.0202
Customer 2 is done with operator 2 at 36.1137
break started for operator 2 at 36.1137
Customer 3 is rooted to operator 2 at 36.1207
Customer 1 is rooted to operator 1 at 38.2832
Customer 1 is assigned to the operator 1 at 38.2832
break ended for operator 2 at 39.1137
Customer 3 is assigned to the operator 2 at 39.1137
Customer 4 initiated a call at 39.8863
Customer 5 initiated a call at 41.1031
Customer 5 is rooted to operator 2 at 41.2097
Customer 3 is done with operator 2 at 45.6809
Customer 5 is assigned to the operator 2 at 45.6809
Customer 6 initiated a call at 45.8921
Customer 4 hangs up the call at 47.3241
Customer 1 is done with operator 1 at 48.4391
break started for operator 1 at 48.4391
Customer 5 is done with operator 2 at 51.0416
break ended for operator 1 at 51

Customer 472 initiated a call at 2698.21
Customer 472 is rooted to operator 2 at 2699.45
Customer 472 is assigned to the operator 2 at 2699.45
Customer 472 is done with operator 2 at 2701.97
Customer 473 initiated a call at 2709.57
Customer 474 initiated a call at 2718.18
Customer 473 is rooted to operator 2 at 2722.75
Customer 473 is assigned to the operator 2 at 2722.75
Customer 475 initiated a call at 2726.4
Customer 476 initiated a call at 2726.98
Customer 473 is done with operator 2 at 2727.02
Customer 475 is rooted to operator 2 at 2727.53
Customer 475 is assigned to the operator 2 at 2727.53
Customer 475 is done with operator 2 at 2728.73
Customer 477 initiated a call at 2728.75
Customer 477 is rooted to operator 1 at 2729.58
Customer 477 is assigned to the operator 1 at 2729.58
Customer 478 initiated a call at 2731.01
Customer 474 is rooted to operator 2 at 2731.18
Customer 474 is assigned to the operator 2 at 2731.18
Customer 474 is done with operator 2 at 2733.55
Customer 476

Customer 772 is rooted to operator 2 at 4485.46
Customer 772 is assigned to the operator 2 at 4485.46
Customer 770 is done with operator 1 at 4485.53
Customer 772 is done with operator 2 at 4492.27
Customer 773 initiated a call at 4492.3
Customer 774 initiated a call at 4494.84
Customer 774 is rooted to operator 2 at 4496.29
Customer 774 is assigned to the operator 2 at 4496.29
Customer 775 initiated a call at 4496.64
Customer 773 hangs up the call at 4496.92
Customer 776 initiated a call at 4497.39
Customer 774 is done with operator 2 at 4500.96
Customer 776 is rooted to operator 2 at 4502.41
Customer 776 is assigned to the operator 2 at 4502.41
Customer 777 initiated a call at 4503.96
Customer 776 is done with operator 2 at 4504.83
break started for operator 2 at 4504.83
Customer 778 initiated a call at 4505.95
Customer 779 initiated a call at 4506.66
Customer 779 is rooted to operator 1 at 4507.2
Customer 779 is assigned to the operator 1 at 4507.2
Customer 775 is rooted to operator

In [11]:
print('Voice Recognition System Utilization: ', router.utilization())

Voice Recognition System Utilization:  0.4205355606426826


In [12]:
print('Utilization Operator 1:', operation_t[0]/FINISHED_T, 
      '\nUtilization Operator 2:' ,operation_t[1]/FINISHED_T)

Utilization Operator 1: 0.4476950984069297 
Utilization Operator 2: 0.43310966445040633


In [13]:
customer_waiting = [c.waiting_t for c in all_customers if c.operator != None]
print('Average Waiting Time: ', sum(customer_waiting)/len(customer_waiting))

Average Waiting Time:  1.605879103793468
